## Extractive Summarization

### Checking the runtime if its connected to GPU (Google Colab Pro Version)

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec  8 17:58:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Checking if the runtime is connected to High RAM (Google Colab Pro Version)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


### Importing the necessary dependencies

In [ ]:
# !pip install pyLDAvis
import nltk
nltk.download('punkt')
from nltk import sent_tokenize,word_tokenize
from nltk import regexp_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet


import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
# import pyLDAvis
# import pyLDAvis.gensim_models as gensimvis
# pyLDAvis.enable_notebook()


import os, re, operator, warnings
warnings.filterwarnings('ignore')
%matplotlib inline


nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/us_president_speeches/presidential_speeches.csv")

### Extracting the rows that has null values in Summary column


In [ ]:
df = df[df['Summary'].notna()]

In [ ]:
# !pip install ohmeow-blurr -q
# !pip install bert-score -q

In [ ]:
# !pip install pandas --upgrade
# !pip install blurr --upgrade

### Importing models for transfer learning

In [ ]:
from fastai.text.all import *
from transformers import *
from blurr.data.all import *
from blurr.modeling.all import *

### Data Cleaning for Summarization

In [ ]:
df['Transcript'] = df['Transcript'].fillna('Not Available')
df['Transcript'] = df['Transcript'].apply(lambda x: x.replace('/',''))
df['Transcript'] = df['Transcript'].apply(lambda x: x.replace('\xa0',''))
df.head()


,Date,President,Party,Speech Title,Summary,Transcript,URL
0,1789-04-30,George Washington,Unaffiliated,First Inaugural Address,"Washington calls on Congress to avoid local and party partisanship and encourages the adoption of a Bill of Rights, without specifically calling them by name. The first President demonstrates his reluctance to accept the post, rejects any salary for the execution of his duties, and devotes a considerable part of the speech to his religious beliefs.","Fellow Citizens of the Senate and the House of Representatives: Among the vicissitudes incident to life, no event could have filled me with greater anxieties than that of which the notification was transmitted by your order, and received on the fourteenth day of the present month. On the one hand, I was summoned by my Country, whose voice I can never hear but with veneration and love, from a retreat which I had chosen with the fondest predilection, and, in my flattering hopes, with an immutable decision, as the asylum of my declining years: a retreat which was rendered every day more neces...",https://millercenter.org/the-presidency/presidential-speeches/april-30-1789-first-inaugural-address
1,1789-10-03,George Washington,Unaffiliated,Thanksgiving Proclamation,"At the request of Congress, Washington establishes November 26th as a national holiday of thanks to God for the successful establishment of the Constitution and government and the liberties the citizens enjoy under the new United States.","Whereas it is the duty of all Nations to acknowledge the providence of Almighty God, to obey his will, to be grateful for his benefits, and humbly to implore his protection and favor, and Whereas both Houses of Congress have by their joint Committee requested me “to recommend to the People of the United States a day of public thanks giving and prayer to be observed by acknowledging with grateful hearts the many signal favors of Almighty God, especially by affording them an opportunity peaceably to establish a form of government for their safety and happiness.” Now therefore I do recommend ...",https://millercenter.org/the-presidency/presidential-speeches/october-3-1789-thanksgiving-proclamation
2,1790-01-08,George Washington,Unaffiliated,First Annual Message to Congress,"In a wide ranging speech, President Washington addresses the need for a regular army, better roads for communication, and taking a census. An important part of his message highlights the need for establishing a good system of education in the country as a way to guarantee all people understood their rights under the Constitution.","Fellow Citizens of the Senate and House of Representatives: I embrace with great satisfaction the opportunity which now presents itself, of congratulating you on the present favourable prospects of our public affairs. The recent accession of the important State of North Carolina to the Constitution of the United States ( of which official information has been received ); the rising credit and respectability of our Country; the general and increasing good will towards the Government of the Union, and the concord, peace, and plenty, with which we are blessed are circumstances auspicious in a...",https://millercenter.org/the-presidency/presidential-speeches/january-8-1790-first-annual-message-congress
3,1790-12-08,George Washington,Unaffiliated,Second Annual Message to Congress,"Washington focuses on commerce in his second address to Congress, outlining the recent loan deal made in Holland and praising the good harvests for maintaining good public credit. The President warns that any type of war on the seas could pose problems for in 1881. commerce, and he encourages developing a reliable form of American transportation of goods to avoid dependence on Europeans in matters of commerce.","Fellow citizens of the Senate and House of Representatives: In meeting you again I feel much satisfaction in being able to repeat my congratulations on the favorable prospects which continu

In [ ]:
articles = df.head(20)

### Downloading the Facebook Bart model for Transfer Learning

In [ ]:

pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, 
                                                                  model_cls=BartForConditionalGeneration)

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

### Hyperparameter Tuning.
* max_len is the input size. Can be increased for better capture. Increasing it might slow down the training time

In [ ]:
hf_batch_tfm = HF_Seq2SeqBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model, task='summarization',
text_gen_kwargs={'max_length': 248,'min_length': 56,'do_sample': False, 'early_stopping': True, 'num_beams': 4, 'temperature': 1.0, 
  'top_k': 50, 'top_p': 1.0, 'repetition_penalty': 1.0, 'bad_words_ids': None, 'bos_token_id': 0, 'pad_token_id': 1,
 'eos_token_id': 2, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'encoder_no_repeat_ngram_size': 0,
 'num_return_sequences': 1, 'decoder_start_token_id': 2, 'use_cache': True, 'num_beam_groups': 1,
 'diversity_penalty': 0.0, 'output_attentions': False, 'output_hidden_states': False, 'output_scores': False,
 'return_dict_in_generate': False, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2, 'remove_invalid_values': False})


blocks = (HF_Seq2SeqBlock(before_batch_tfm=hf_batch_tfm), noop)

dblock = DataBlock(blocks=blocks, get_x=ColReader('Transcript'), splitter=RandomSplitter())

In [ ]:
dls = dblock.dataloaders(articles, bs=2)

### Defining all the evaluation metrics

In [ ]:
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]
        }
    }

#### Freezing all the hidden layers except the last layer

In [ ]:
model = HF_BaseModelWrapper(hf_model)
learn_cbs = [HF_BaseModelCallback]
fit_cbs = [HF_Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

learn = Learner(dls, 
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,
                splitter=partial(seq2seq_splitter, arch=hf_arch)).to_fp16()

learn.create_opt() 
learn.freeze()

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

#### One cycle training. Uses large learning rates to make the model converge faster

In [ ]:
learn.fit_one_cycle(40, lr_max=3e-5, cbs=fit_cbs)

epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,bertscore_precision,bertscore_recall,bertscore_f1,time
0,6.270509,6.442061,0.000000,0.000000,0.000000,0.477098,0.632300,0.543504,00:37
1,6.157137,6.198542,0.000000,0.000000,0.000000,0.457596,0.596107,0.517572,00:10
2,6.059958,5.783450,0.000000,0.000000,0.000000,0.464319,0.603798,0.524801,00:10
3,5.772096,5.026595,0.000000,0.000000,0.000000,0.463611,0.598815,0.522435,00:09
4,5.200620,2.762472,0.000000,0.000000,0.000000,0.474280,0.601807,0.530391,00:09
5,4.435616,1.474391,0.000000,0.000000,0.000000,0.487015,0.617111,0.543333,00:08
6,3.681740,1.004901,0.000000,0.000000,0.000000,0.488797,0.618168,0.544821,00:08
7,3.035388,0.445885,0.050000,0.026334,0.050000,0.540503,0.687602,0.604062,00:07
8,2.519478,0.700701,0.033319,0.012821,0.033319,0.548484,0.691929,0.611577,00:08
9,2.111110,0.500573,0.019133,0.000000,0.019133,0.540333,0.669149,0.597621,00:08


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

In [ ]:
df['Transcript'].iloc[28]


"I have received the message in writing which you sent me through Captain Irvine, our confidential agent, placed near you for the purpose of communicating and transacting between us, whatever may be useful for both nations. I am happy to learn you have been so far favored by the Divine spirit as to be made sensible of those things which are for your good and that of your people, and of those which are hurtful to you; and particularly that you and they see the ruinous effects which the abuse of spirituous liquors have produced upon them. It has weakened their bodies, enervated their minds, exposed them to hunger, cold, nakedness, and poverty, kept them in perpetual broils, and reduced their population. I do not wonder then, brother, at your censures, not only on your own people, who have voluntarily gone into these fatal habits, but on all the nations of white people who have supplied their calls for this article. But these nations have done to you only what they do among themselves. Th

#### Generating Summaries for the entire dataset

In [ ]:
president_options = df['President'].value_counts().keys()
print(len(president_options))

40


In [ ]:
from google.colab import files
president_summary = []
original_summary = []
from tqdm import tqdm
for president in tqdm(president_options):
  df_president = df.loc[df['President'] == president]
  for i,j in zip(df_president['Transcript'],df_president['Summary']):
    outputs = learn.blurr_generate(i, early_stopping=False, num_return_sequences=1)
    if (len(j) == 0):
      j = "Summary Not Available in Dataset"
    president_summary.append(outputs)
    original_summary.append(j)

  df_output = pd.DataFrame()
  df_output['Generated_Summary'] = president_summary
  df_output['Original_Summary'] = original_summary 
  text_string = president + '.csv'
  df_output.to_csv(text_string)
  files.download(text_string)
  print("Done for :",president)
  president_summary = []
  original_summary = []




  0%|          | 0/40 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  2%|▎         | 1/40 [02:12<1:26:16, 132.72s/it]

Done for : Lyndon B. Johnson


...

Done for : William Harrison


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

100%|██████████| 40/40 [24:05<00:00, 36.15s/it]

Done for : Millard Fillmore


The above files have summaries for each of teh president. This is done to ensure smooth running of the Front end UI